# LangChain Router Chain + Validators for Fraud Detection

This notebook demonstrates a **fraud detection workflow** using LangChain with:

- **Router Chain** - Routes transactions to specialized processing chains based on transaction type
- **Pydantic Validators** - Strict input/output validation for type safety and guardrails
- **Structured Output** - JSON output parsing with schema validation

The focus is on:
- Dynamic routing based on transaction characteristics
- Input validation before LLM processing
- Output validation to ensure safe, structured responses
- Type-safe data flow throughout the pipeline

## Architecture Overview

This workflow has the following components:

1. **Input Validator** - Validates transaction data using Pydantic before processing
2. **Router Chain** - Routes to specialized chains based on transaction type
3. **Specialized Chains** - Different analysis chains for different transaction types
4. **Output Validator** - Validates LLM output against a strict schema
5. **Safety Validator** - Ensures no forbidden actions in the response

**Key principle:** Route to the right expert, validate everything.

### Workflow Diagram

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                    ROUTER CHAIN + VALIDATORS WORKFLOW                       │
└─────────────────────────────────────────────────────────────────────────────┘

                              ┌───────────────┐
                              │  Transaction  │
                              │    Input      │
                              └───────┬───────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│  STEP 1: INPUT VALIDATION                                                   │
│  ┌────────────────────────────────────────────────────────────────────┐     │
│  │ Pydantic TransactionInput Model                                    │     │
│  │ • Validates required fields (id, amount, currency, type)           │     │
│  │ • Validates amount > 0                                             │     │
│  │ • Validates currency is valid ISO code                             │     │
│  │ • Validates transaction_type is known                              │     │
│  └────────────────────────────────────────────────────────────────────┘     │
└─────────────────────────────────────────────────────────────────────────────┘
                                      │
                                      ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│  STEP 2: ROUTER CHAIN                                                       │
│  ┌────────────────────────────────────────────────────────────────────┐     │
│  │ RunnableBranch - Routes based on transaction_type                  │     │
│  └────────────────────────────────────────────────────────────────────┘     │
└─────────────────────────────────────────────────────────────────────────────┘
                                      │
                 ┌────────────────────┼────────────────────┐
                 │                    │                    │
                 ▼                    ▼                    ▼
        ┌───────────────┐    ┌───────────────┐    ┌───────────────┐
        │   WIRE        │    │    CARD       │    │   CRYPTO      │
        │   TRANSFER    │    │   PAYMENT     │    │   PAYMENT     │
        │   CHAIN       │    │   CHAIN       │    │   CHAIN       │
        └───────┬───────┘    └───────┬───────┘    └───────┬───────┘
                │                    │                    │
                └────────────────────┼────────────────────┘
                                     │
                                     ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│  STEP 3: OUTPUT VALIDATION                                                  │
│  ┌────────────────────────────────────────────────────────────────────┐     │
│  │ Pydantic FraudAnalysisOutput Model                                 │     │
│  │ • Validates risk_score is 0.0-1.0                                  │     │
│  │ • Validates fraud_indicators is a list                             │     │
│  │ • Validates recommendation is present                              │     │
│  └────────────────────────────────────────────────────────────────────┘     │
└─────────────────────────────────────────────────────────────────────────────┘
                                     │
                                     ▼
┌─────────────────────────────────────────────────────────────────────────────┐
│  STEP 4: SAFETY VALIDATION                                                  │
│  ┌────────────────────────────────────────────────────────────────────┐     │
│  │ SafetyValidator                                                    │     │
│  │ • Checks for forbidden action words                                │     │
│  │ • Ensures AI only recommends, never decides                        │     │
│  └────────────────────────────────────────────────────────────────────┘     │
└─────────────────────────────────────────────────────────────────────────────┘
                                     │
                                     ▼
                          ┌───────────────────────┐
                          │   Analyst Dashboard   │
                          │   (Human Review)      │
                          └───────────────────────┘
```

## 1. Install Dependencies

Install LangChain and Ollama packages. Ollama runs locally, no API key needed.

**Prerequisites**: You need Ollama installed and running with a model downloaded.
```bash
ollama pull llama3.1:8b
```

In [22]:
!pip install langchain langchain-ollama pydantic


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


## 2. Import Libraries

In [23]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnablePassthrough
from langchain_ollama import ChatOllama
from pydantic import BaseModel, Field, field_validator, model_validator
from typing import List, Optional, Literal
from enum import Enum
import json
from datetime import datetime

## 3. Define Input Validators (Pydantic Models)

Pydantic models ensure that all input data is validated before reaching the LLM. This prevents malformed data from causing issues downstream.

In [24]:
class TransactionType(str, Enum):
    WIRE_TRANSFER = "wire_transfer"
    CARD_PAYMENT = "card_payment"
    CRYPTO_PAYMENT = "crypto_payment"
    ACH_TRANSFER = "ach_transfer"

VALID_CURRENCIES = ["USD", "EUR", "GBP", "BRL", "JPY", "CAD", "AUD", "BTC", "ETH"]

class TransactionInput(BaseModel):
    """Validates incoming transaction data before processing."""
    
    transaction_id: str = Field(..., min_length=1, description="Unique transaction identifier")
    amount: float = Field(..., gt=0, description="Transaction amount (must be positive)")
    currency: str = Field(..., description="ISO currency code")
    transaction_type: TransactionType = Field(..., description="Type of transaction")
    sender_country: str = Field(..., min_length=2, max_length=3, description="Sender country code")
    receiver_country: str = Field(..., min_length=2, max_length=3, description="Receiver country code")
    sender_account_age_days: int = Field(..., ge=0, description="Account age in days")
    is_first_transaction: bool = Field(default=False, description="Whether this is the first transaction")
    device_fingerprint: Optional[str] = Field(default=None, description="Device fingerprint if available")
    ip_country: Optional[str] = Field(default=None, description="IP geolocation country")
    geo_mismatch: bool = Field(default=False, description="Whether IP country mismatches sender country")

    @field_validator('currency')
    @classmethod
    def validate_currency(cls, v: str) -> str:
        if v.upper() not in VALID_CURRENCIES:
            raise ValueError(f"Invalid currency: {v}. Must be one of {VALID_CURRENCIES}")
        return v.upper()

    @model_validator(mode='after')
    def validate_geo_mismatch(self):
        if self.ip_country and self.sender_country:
            object.__setattr__(self, 'geo_mismatch', self.ip_country != self.sender_country)
        return self

print("TransactionInput validator defined!")

TransactionInput validator defined!


### Test Input Validation

In [25]:
valid_transaction = TransactionInput(
    transaction_id="TX001",
    amount=5000.00,
    currency="USD",
    transaction_type=TransactionType.WIRE_TRANSFER,
    sender_country="US",
    receiver_country="BR",
    sender_account_age_days=30,
    is_first_transaction=True,
    ip_country="US"
)
print("✅ Valid transaction created:")
print(f"   ID: {valid_transaction.transaction_id}")
print(f"   Amount: {valid_transaction.amount} {valid_transaction.currency}")
print(f"   Type: {valid_transaction.transaction_type.value}")

try:
    invalid_transaction = TransactionInput(
        transaction_id="TX002",
        amount=-100,
        currency="INVALID",
        transaction_type=TransactionType.CARD_PAYMENT,
        sender_country="US",
        receiver_country="CA",
        sender_account_age_days=10
    )
except Exception as e:
    print(f"\n❌ Invalid transaction rejected: {e}")

✅ Valid transaction created:
   ID: TX001
   Amount: 5000.0 USD
   Type: wire_transfer

❌ Invalid transaction rejected: 2 validation errors for TransactionInput
amount
  Input should be greater than 0 [type=greater_than, input_value=-100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than
currency
  Value error, Invalid currency: INVALID. Must be one of ['USD', 'EUR', 'GBP', 'BRL', 'JPY', 'CAD', 'AUD', 'BTC', 'ETH'] [type=value_error, input_value='INVALID', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error


## 4. Define Output Validators

Output validators ensure the LLM response conforms to our expected schema.

In [26]:
class FraudIndicator(BaseModel):
    """A single fraud indicator with severity."""
    indicator: str = Field(..., description="Description of the fraud indicator")
    severity: Literal["low", "medium", "high"] = Field(..., description="Severity level")
    confidence: float = Field(..., ge=0.0, le=1.0, description="Confidence score")

class FraudAnalysisOutput(BaseModel):
    """Validates the fraud analysis output from the LLM."""
    
    transaction_id: str = Field(..., description="Transaction ID analyzed")
    risk_score: float = Field(..., ge=0.0, le=1.0, description="Overall risk score 0-1")
    risk_level: Literal["low", "medium", "high", "critical"] = Field(..., description="Risk level category")
    fraud_indicators: List[FraudIndicator] = Field(default_factory=list, description="List of fraud indicators")
    analysis_summary: str = Field(..., min_length=10, description="Summary of the analysis")
    recommended_action: Literal[
        "proceed_with_monitoring",
        "request_additional_verification",
        "escalate_to_analyst",
        "flag_for_review"
    ] = Field(..., description="Recommended next action")

    @field_validator('risk_level')
    @classmethod
    def validate_risk_level_matches_score(cls, v, info):
        return v

print("FraudAnalysisOutput validator defined!")

FraudAnalysisOutput validator defined!


## 5. Define Safety Validator

The safety validator ensures the LLM output doesn't contain forbidden action words that could imply the AI is making payment decisions.

In [27]:
class SafetyValidator:
    """Validates that LLM output doesn't contain forbidden actions."""
    
    FORBIDDEN_PHRASES = [
        "approve the transaction",
        "reject the transaction",
        "block the payment",
        "cancel the transaction",
        "authorize payment",
        "deny the transfer",
        "i have decided",
        "i will approve",
        "i will reject"
    ]
    
    @classmethod
    def validate(cls, text: str) -> dict:
        text_lower = text.lower()
        violations = [phrase for phrase in cls.FORBIDDEN_PHRASES if phrase in text_lower]
        return {
            "is_safe": len(violations) == 0,
            "violations": violations,
            "message": "Output is safe" if not violations else f"Forbidden phrases detected: {violations}"
        }

print("SafetyValidator defined!")
print("\nTest safe text:", SafetyValidator.validate("The transaction shows medium risk."))
print("Test unsafe text:", SafetyValidator.validate("I have decided to approve the transaction."))

SafetyValidator defined!

Test safe text: {'is_safe': True, 'violations': [], 'message': 'Output is safe'}
Test unsafe text: {'is_safe': False, 'violations': ['approve the transaction', 'i have decided'], 'message': "Forbidden phrases detected: ['approve the transaction', 'i have decided']"}


## 6. Setup LLM

Configure Ollama with Llama 3.1 8B model.

In [28]:
llm = ChatOllama(model='llama3.1:8b', temperature=0)
print(f"LLM configured: {llm.model}")

LLM configured: llama3.1:8b


## 7. Define Specialized Chains

Each transaction type has a specialized chain with domain-specific prompts.

In [29]:
wire_transfer_prompt = PromptTemplate(
    input_variables=['transaction'],
    template="""
You are a wire transfer fraud analyst assistant. Analyze the following wire transfer for fraud indicators.
Focus on: cross-border risks, amount thresholds, account age, and geographic patterns.

IMPORTANT: You are an assistant. You do NOT approve, reject, or make decisions. You only analyze and recommend.

Transaction: {transaction}

Provide your analysis as JSON with these fields:
- transaction_id: the transaction ID
- risk_score: a float between 0.0 and 1.0
- risk_level: one of "low", "medium", "high", "critical"
- fraud_indicators: array of objects with "indicator", "severity", "confidence"
- analysis_summary: a summary of your findings
- recommended_action: one of "proceed_with_monitoring", "request_additional_verification", "escalate_to_analyst", "flag_for_review"

Respond with valid JSON only.
"""
)

card_payment_prompt = PromptTemplate(
    input_variables=['transaction'],
    template="""
You are a card payment fraud analyst assistant. Analyze the following card transaction for fraud indicators.
Focus on: unusual spending patterns, geographic anomalies, device fingerprints, and velocity checks.

IMPORTANT: You are an assistant. You do NOT approve, reject, or make decisions. You only analyze and recommend.

Transaction: {transaction}

Provide your analysis as JSON with these fields:
- transaction_id: the transaction ID
- risk_score: a float between 0.0 and 1.0
- risk_level: one of "low", "medium", "high", "critical"
- fraud_indicators: array of objects with "indicator", "severity", "confidence"
- analysis_summary: a summary of your findings
- recommended_action: one of "proceed_with_monitoring", "request_additional_verification", "escalate_to_analyst", "flag_for_review"

Respond with valid JSON only.
"""
)

crypto_payment_prompt = PromptTemplate(
    input_variables=['transaction'],
    template="""
You are a cryptocurrency fraud analyst assistant. Analyze the following crypto transaction for fraud indicators.
Focus on: wallet reputation, transaction mixing patterns, regulatory compliance, and AML red flags.

IMPORTANT: You are an assistant. You do NOT approve, reject, or make decisions. You only analyze and recommend.

Transaction: {transaction}

Provide your analysis as JSON with these fields:
- transaction_id: the transaction ID
- risk_score: a float between 0.0 and 1.0
- risk_level: one of "low", "medium", "high", "critical"
- fraud_indicators: array of objects with "indicator", "severity", "confidence"
- analysis_summary: a summary of your findings
- recommended_action: one of "proceed_with_monitoring", "request_additional_verification", "escalate_to_analyst", "flag_for_review"

Respond with valid JSON only.
"""
)

default_prompt = PromptTemplate(
    input_variables=['transaction'],
    template="""
You are a payment fraud analyst assistant. Analyze the following transaction for fraud indicators.

IMPORTANT: You are an assistant. You do NOT approve, reject, or make decisions. You only analyze and recommend.

Transaction: {transaction}

Provide your analysis as JSON with these fields:
- transaction_id: the transaction ID
- risk_score: a float between 0.0 and 1.0
- risk_level: one of "low", "medium", "high", "critical"
- fraud_indicators: array of objects with "indicator", "severity", "confidence"
- analysis_summary: a summary of your findings
- recommended_action: one of "proceed_with_monitoring", "request_additional_verification", "escalate_to_analyst", "flag_for_review"

Respond with valid JSON only.
"""
)

wire_chain = wire_transfer_prompt | llm | StrOutputParser()
card_chain = card_payment_prompt | llm | StrOutputParser()
crypto_chain = crypto_payment_prompt | llm | StrOutputParser()
default_chain = default_prompt | llm | StrOutputParser()

print("Specialized chains created:")
print("  - wire_chain (for wire transfers)")
print("  - card_chain (for card payments)")
print("  - crypto_chain (for crypto payments)")
print("  - default_chain (fallback)")

Specialized chains created:
  - wire_chain (for wire transfers)
  - card_chain (for card payments)
  - crypto_chain (for crypto payments)
  - default_chain (fallback)


## 8. Define Router Chain

The router chain uses `RunnableBranch` to route transactions to the appropriate specialized chain based on transaction type.

In [30]:
def get_transaction_type(inputs: dict) -> str:
    """Extract transaction type from inputs."""
    if isinstance(inputs.get('transaction'), TransactionInput):
        return inputs['transaction'].transaction_type.value
    elif isinstance(inputs.get('transaction'), dict):
        return inputs['transaction'].get('transaction_type', 'unknown')
    return 'unknown'

def prepare_for_chain(inputs: dict) -> dict:
    """Prepare transaction for chain processing."""
    tx = inputs.get('transaction')
    if isinstance(tx, TransactionInput):
        return {'transaction': tx.model_dump_json()}
    elif isinstance(tx, dict):
        return {'transaction': json.dumps(tx)}
    return {'transaction': str(tx)}

router_chain = RunnableBranch(
    (
        lambda x: get_transaction_type(x) == TransactionType.WIRE_TRANSFER.value,
        RunnableLambda(prepare_for_chain) | wire_chain
    ),
    (
        lambda x: get_transaction_type(x) == TransactionType.CARD_PAYMENT.value,
        RunnableLambda(prepare_for_chain) | card_chain
    ),
    (
        lambda x: get_transaction_type(x) == TransactionType.CRYPTO_PAYMENT.value,
        RunnableLambda(prepare_for_chain) | crypto_chain
    ),
    RunnableLambda(prepare_for_chain) | default_chain
)

print("Router chain created with branches:")
print("  - wire_transfer -> wire_chain")
print("  - card_payment -> card_chain")
print("  - crypto_payment -> crypto_chain")
print("  - default -> default_chain")

Router chain created with branches:
  - wire_transfer -> wire_chain
  - card_payment -> card_chain
  - crypto_payment -> crypto_chain
  - default -> default_chain


## 9. Define Test Transactions

Create sample transactions for each type to test the router.

In [31]:
wire_transaction = TransactionInput(
    transaction_id="WIRE001",
    amount=25000.00,
    currency="USD",
    transaction_type=TransactionType.WIRE_TRANSFER,
    sender_country="US",
    receiver_country="NG",
    sender_account_age_days=15,
    is_first_transaction=True,
    ip_country="RU"
)

card_transaction = TransactionInput(
    transaction_id="CARD001",
    amount=500.00,
    currency="EUR",
    transaction_type=TransactionType.CARD_PAYMENT,
    sender_country="DE",
    receiver_country="DE",
    sender_account_age_days=365,
    is_first_transaction=False,
    device_fingerprint="fp_abc123",
    ip_country="DE"
)

crypto_transaction = TransactionInput(
    transaction_id="CRYPTO001",
    amount=10000.00,
    currency="BTC",
    transaction_type=TransactionType.CRYPTO_PAYMENT,
    sender_country="US",
    receiver_country="KY",
    sender_account_age_days=5,
    is_first_transaction=True
)

print("Test transactions created:")
print(f"  1. Wire Transfer: {wire_transaction.transaction_id} - ${wire_transaction.amount} to {wire_transaction.receiver_country}")
print(f"  2. Card Payment: {card_transaction.transaction_id} - €{card_transaction.amount}")
print(f"  3. Crypto Payment: {crypto_transaction.transaction_id} - {crypto_transaction.amount} BTC")

Test transactions created:
  1. Wire Transfer: WIRE001 - $25000.0 to NG
  2. Card Payment: CARD001 - €500.0
  3. Crypto Payment: CRYPTO001 - 10000.0 BTC


## 10. Execute Router Chain - Wire Transfer

Test the router with a suspicious wire transfer.

In [32]:
print("=" * 60)
print("ROUTING: Wire Transfer Transaction")
print("=" * 60)
print(f"\nTransaction Type: {wire_transaction.transaction_type.value}")
print(f"Expected Route: wire_chain")
print("\nProcessing...")

wire_result = router_chain.invoke({'transaction': wire_transaction})
print("\nRaw LLM Output:")
print(wire_result)

ROUTING: Wire Transfer Transaction

Transaction Type: wire_transfer
Expected Route: wire_chain

Processing...

Raw LLM Output:
```json
{
  "transaction_id": "WIRE001",
  "risk_score": 0.8,
  "risk_level": "high",
  "fraud_indicators": [
    {
      "indicator": "cross-border risk",
      "severity": "medium",
      "confidence": 0.7
    },
    {
      "indicator": "amount threshold exceeded",
      "severity": "high",
      "confidence": 0.9
    },
    {
      "indicator": "geo-mismatch detected",
      "severity": "low",
      "confidence": 0.4
    }
  ],
  "analysis_summary": "This transaction exhibits high risk due to a large amount being transferred across borders, with the sender's account being relatively new and a geo-mismatch detected.",
  "recommended_action": "request_additional_verification"
}
```


## 11. Validate and Parse Output

In [33]:
def extract_json_from_response(response: str) -> dict:
    """Extract JSON from LLM response that might include markdown code blocks."""
    import re
    json_match = re.search(r'```(?:json)?\s*([\s\S]*?)```', response)
    if json_match:
        json_str = json_match.group(1).strip()
    else:
        json_str = response.strip()
    return json.loads(json_str)

def validate_and_parse_output(raw_output: str, transaction_id: str) -> dict:
    """Validate LLM output against schema and safety rules."""
    result = {
        'success': False,
        'parsed_output': None,
        'safety_check': None,
        'errors': []
    }
    
    safety_check = SafetyValidator.validate(raw_output)
    result['safety_check'] = safety_check
    if not safety_check['is_safe']:
        result['errors'].append(f"Safety violation: {safety_check['violations']}")
        return result
    
    try:
        parsed_json = extract_json_from_response(raw_output)
        parsed_json['transaction_id'] = transaction_id
        validated_output = FraudAnalysisOutput(**parsed_json)
        result['parsed_output'] = validated_output
        result['success'] = True
    except json.JSONDecodeError as e:
        result['errors'].append(f"JSON parsing error: {e}")
    except Exception as e:
        result['errors'].append(f"Validation error: {e}")
    
    return result

print("Output validation functions defined!")

Output validation functions defined!


In [34]:
print("=" * 60)
print("VALIDATING: Wire Transfer Output")
print("=" * 60)

wire_validation = validate_and_parse_output(wire_result, wire_transaction.transaction_id)

print(f"\n✅ Safety Check: {wire_validation['safety_check']['message']}")

if wire_validation['success']:
    output = wire_validation['parsed_output']
    print(f"\n✅ Schema Validation: Passed")
    print(f"\nParsed Output:")
    print(f"  Transaction ID: {output.transaction_id}")
    print(f"  Risk Score: {output.risk_score}")
    print(f"  Risk Level: {output.risk_level}")
    print(f"  Recommended Action: {output.recommended_action}")
    print(f"  Summary: {output.analysis_summary}")
    print(f"  Fraud Indicators: {len(output.fraud_indicators)}")
    for indicator in output.fraud_indicators:
        print(f"    - [{indicator.severity}] {indicator.indicator} (conf: {indicator.confidence})")
else:
    print(f"\n❌ Validation Errors: {wire_validation['errors']}")

VALIDATING: Wire Transfer Output

✅ Safety Check: Output is safe

✅ Schema Validation: Passed

Parsed Output:
  Transaction ID: WIRE001
  Risk Score: 0.8
  Risk Level: high
  Recommended Action: request_additional_verification
  Summary: This transaction exhibits high risk due to a large amount being transferred across borders, with the sender's account being relatively new and a geo-mismatch detected.
  Fraud Indicators: 3
    - [medium] cross-border risk (conf: 0.7)
    - [high] amount threshold exceeded (conf: 0.9)
    - [low] geo-mismatch detected (conf: 0.4)


## 12. Execute Router Chain - Card Payment

In [35]:
print("=" * 60)
print("ROUTING: Card Payment Transaction")
print("=" * 60)
print(f"\nTransaction Type: {card_transaction.transaction_type.value}")
print(f"Expected Route: card_chain")
print("\nProcessing...")

card_result = router_chain.invoke({'transaction': card_transaction})
card_validation = validate_and_parse_output(card_result, card_transaction.transaction_id)

print(f"\n✅ Safety Check: {card_validation['safety_check']['message']}")

if card_validation['success']:
    output = card_validation['parsed_output']
    print(f"✅ Schema Validation: Passed")
    print(f"\nParsed Output:")
    print(f"  Risk Score: {output.risk_score}")
    print(f"  Risk Level: {output.risk_level}")
    print(f"  Recommended Action: {output.recommended_action}")
else:
    print(f"\n❌ Validation Errors: {card_validation['errors']}")

ROUTING: Card Payment Transaction

Transaction Type: card_payment
Expected Route: card_chain

Processing...

✅ Safety Check: Output is safe
✅ Schema Validation: Passed

Parsed Output:
  Risk Score: 0.2
  Risk Level: low
  Recommended Action: proceed_with_monitoring


## 13. Execute Router Chain - Crypto Payment

In [36]:
print("=" * 60)
print("ROUTING: Crypto Payment Transaction")
print("=" * 60)
print(f"\nTransaction Type: {crypto_transaction.transaction_type.value}")
print(f"Expected Route: crypto_chain")
print("\nProcessing...")

crypto_result = router_chain.invoke({'transaction': crypto_transaction})
crypto_validation = validate_and_parse_output(crypto_result, crypto_transaction.transaction_id)

print(f"\n✅ Safety Check: {crypto_validation['safety_check']['message']}")

if crypto_validation['success']:
    output = crypto_validation['parsed_output']
    print(f"✅ Schema Validation: Passed")
    print(f"\nParsed Output:")
    print(f"  Risk Score: {output.risk_score}")
    print(f"  Risk Level: {output.risk_level}")
    print(f"  Recommended Action: {output.recommended_action}")
else:
    print(f"\n❌ Validation Errors: {crypto_validation['errors']}")

ROUTING: Crypto Payment Transaction

Transaction Type: crypto_payment
Expected Route: crypto_chain

Processing...

✅ Safety Check: Output is safe
✅ Schema Validation: Passed

Parsed Output:
  Risk Score: 0.7
  Risk Level: medium
  Recommended Action: request_additional_verification


## 14. Complete Pipeline Function

Wrap the entire workflow in a function with full validation.

In [37]:
def analyze_transaction(transaction_data: dict) -> dict:
    """
    Complete fraud analysis pipeline with input/output validation.
    
    Args:
        transaction_data: Raw transaction dictionary
        
    Returns:
        Analysis result with validation status
    """
    result = {
        'timestamp': datetime.now().isoformat(),
        'input_validation': {'success': False},
        'routing': None,
        'analysis': None,
        'output_validation': None,
        'final_output': None
    }
    
    try:
        validated_input = TransactionInput(**transaction_data)
        result['input_validation'] = {
            'success': True,
            'transaction_id': validated_input.transaction_id
        }
    except Exception as e:
        result['input_validation'] = {
            'success': False,
            'error': str(e)
        }
        return result
    
    result['routing'] = {
        'transaction_type': validated_input.transaction_type.value,
        'routed_to': f"{validated_input.transaction_type.value}_chain"
    }
    
    raw_analysis = router_chain.invoke({'transaction': validated_input})
    result['analysis'] = {'raw_output': raw_analysis}
    
    validation = validate_and_parse_output(raw_analysis, validated_input.transaction_id)
    result['output_validation'] = {
        'success': validation['success'],
        'safety_passed': validation['safety_check']['is_safe'],
        'errors': validation['errors']
    }
    
    if validation['success']:
        result['final_output'] = validation['parsed_output'].model_dump()
    
    return result

print("Complete pipeline function created!")

Complete pipeline function created!


In [38]:
print("=" * 60)
print("TESTING COMPLETE PIPELINE")
print("=" * 60)

test_data = {
    'transaction_id': 'TEST001',
    'amount': 15000.00,
    'currency': 'USD',
    'transaction_type': 'wire_transfer',
    'sender_country': 'US',
    'receiver_country': 'RU',
    'sender_account_age_days': 7,
    'is_first_transaction': True,
    'ip_country': 'CN'
}

pipeline_result = analyze_transaction(test_data)

print(f"\nTimestamp: {pipeline_result['timestamp']}")
print(f"\nInput Validation: {'✅ Passed' if pipeline_result['input_validation']['success'] else '❌ Failed'}")
print(f"Routing: {pipeline_result['routing']}")
print(f"Output Validation: {'✅ Passed' if pipeline_result['output_validation']['success'] else '❌ Failed'}")

if pipeline_result['final_output']:
    print(f"\n📊 Final Analysis:")
    print(json.dumps(pipeline_result['final_output'], indent=2))

TESTING COMPLETE PIPELINE

Timestamp: 2025-12-19T15:56:30.792473

Input Validation: ✅ Passed
Routing: {'transaction_type': 'wire_transfer', 'routed_to': 'wire_transfer_chain'}
Output Validation: ✅ Passed

📊 Final Analysis:
{
  "transaction_id": "TEST001",
  "risk_score": 0.85,
  "risk_level": "high",
  "fraud_indicators": [
    {
      "indicator": "cross-border risk",
      "severity": "medium",
      "confidence": 0.7
    },
    {
      "indicator": "amount threshold exceeded",
      "severity": "high",
      "confidence": 0.9
    },
    {
      "indicator": "geo-mismatch detected",
      "severity": "low",
      "confidence": 0.4
    }
  ],
  "analysis_summary": "This transaction exhibits high risk due to a large amount being transferred across borders, with the sender's account being only 7 days old and having no prior transactions. Additionally, there is a geo-mismatch detected between the IP country (CN) and the sender's country (US).",
  "recommended_action": "request_additional

## 15. Audit Logging

In [39]:
audit_log = {
    'timestamp': datetime.now().isoformat(),
    'workflow': 'router_validators_fraud_detection',
    'transactions_processed': [
        {
            'id': wire_transaction.transaction_id,
            'type': wire_transaction.transaction_type.value,
            'routed_to': 'wire_chain',
            'validation_passed': wire_validation['success']
        },
        {
            'id': card_transaction.transaction_id,
            'type': card_transaction.transaction_type.value,
            'routed_to': 'card_chain',
            'validation_passed': card_validation['success']
        },
        {
            'id': crypto_transaction.transaction_id,
            'type': crypto_transaction.transaction_type.value,
            'routed_to': 'crypto_chain',
            'validation_passed': crypto_validation['success']
        }
    ],
    'metadata': {
        'llm_model': 'llama3.1:8b',
        'router_type': 'RunnableBranch',
        'validators': ['TransactionInput', 'FraudAnalysisOutput', 'SafetyValidator']
    }
}

print("Audit Log:")
print(json.dumps(audit_log, indent=2))

Audit Log:
{
  "timestamp": "2025-12-19T15:56:35.569169",
  "workflow": "router_validators_fraud_detection",
  "transactions_processed": [
    {
      "id": "WIRE001",
      "type": "wire_transfer",
      "routed_to": "wire_chain",
      "validation_passed": true
    },
    {
      "id": "CARD001",
      "type": "card_payment",
      "routed_to": "card_chain",
      "validation_passed": true
    },
    {
      "id": "CRYPTO001",
      "type": "crypto_payment",
      "routed_to": "crypto_chain",
      "validation_passed": true
    }
  ],
  "metadata": {
    "llm_model": "llama3.1:8b",
    "router_type": "RunnableBranch",
    "validators": [
      "TransactionInput",
      "FraudAnalysisOutput",
      "SafetyValidator"
    ]
  }
}


## Summary

This notebook demonstrated:

### Router Chain Pattern
- **`RunnableBranch`** for conditional routing based on transaction type
- **Specialized chains** for wire transfers, card payments, and crypto
- **Dynamic routing** that selects the right expert for each transaction

### Validator Pattern
- **Input Validators** (`TransactionInput`) - Pydantic models with field validators
- **Output Validators** (`FraudAnalysisOutput`) - Structured output validation
- **Safety Validators** (`SafetyValidator`) - Guardrails against forbidden actions

### Key Benefits
1. **Type Safety** - Pydantic ensures data integrity throughout the pipeline
2. **Modularity** - Specialized chains can be updated independently
3. **Auditability** - Every step is logged and validated
4. **Safety** - Forbidden phrases are caught before reaching the analyst

### Comparison with Sequential Chain

| Feature | Sequential Chain | Router Chain |
|---------|-----------------|---------------|
| Flow | Linear A → B → C | Conditional A → {B1, B2, B3} |
| Use Case | Fixed workflow | Domain-specific routing |
| Flexibility | Low | High |
| Complexity | Simple | Medium |

### Extensions
- Add more transaction types (ACH, SEPA, etc.)
- Implement confidence-based re-routing
- Add human-in-the-loop for critical risk levels
- Integrate with real fraud detection databases